In [35]:
def get_top100_list():
    output_list = list()
    url = "https://www.billboard.com/charts/hot-100"

    import requests
    from bs4 import BeautifulSoup
    
    try:
        response = requests.get(url)   
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                soup = BeautifulSoup(response.content,'lxml')
            except:
                print('something went wrong')
    except:
        print("Something went wrong with request.get")
    
   # all_songs = soup.find_all('span',class_='chart-list-item__title-text')
    all_songs = soup.find_all('div',class_='chart-list-item__text chart-list-item__text--has-video')
    
    for song in all_songs:
        song_name = song.find('span',class_='chart-list-item__title-text').get_text()
        artist = song.find('div',class_='chart-list-item__artist').get_text()
        try:
            lyrics_link = song.find('div',class_='chart-list-item__lyrics').find('a').get('href')
        except:
            lyrics_link = 'no_link_for_the_lyrics_on_billboard'

        if song_name:
            try:
                lyrics = get_lyrics(lyrics_link)
            except:
                lyrics = 'no_link_for_the_lyrics_on_billboard'
            output_list.append((song_name,artist,lyrics))
    
    return output_list

In [36]:
get_top100_list()

[('\nSicko Mode\n',
  '\n\nTravis Scott\n\n',
  'Astro, yeahSun is down, freezin\' coldThat\'s how we already know winter\'s hereMy dawg would prolly do it for a Louis beltThat\'s just all he know, he don\'t know nothin\' elseI tried to show \'em, yeahI tried to show \'em, yeah, yeahYeah, yeah, yeahGone on you with the pick and rollYoung LaFlame, he in sicko modeWoo, made this here with all the ice on in the boothAt the gate outside, when they pull up, they get me looseYeah, Jump Out boys, that\'s Nike boys, hoppin\' out coupesThis shit way too big, when we pull up give me the loot(Gimme the loot!)Was off the Remy, had a PapooseHad to hit my old town to duck the newsTwo-four hour lockdown, we made no movesNow it\'s 4AM and I\'m back up poppin\' with the crewI just landed in, Chase B mixes pop like Jamba JuiceDifferent colored chains, think my jeweler really sellin\' fruitsAnd they chokin\', man, know the crackers wish it was a nooseSome-some-some, someone saidTo win the retreat, we all

In [31]:
def get_lyrics(lyrics_link):
    import requests
    from bs4 import BeautifulSoup 
    page = requests.get(lyrics_link)
    soup = BeautifulSoup(page.content,'lxml') 
    
    try:
        lyrics = soup.find('div',class_='lyrics').find('p').get_text() 
    except:
        lyrics = '--'
    
    return lyrics

In [29]:
#test get_lyrics
get_lyrics('https://www.billboard.com/articles/news/lyrics/8477102/travis-scott-sicko-mode-lyrics')

'Astro, yeahSun is down, freezin\' coldThat\'s how we already know winter\'s hereMy dawg would prolly do it for a Louis beltThat\'s just all he know, he don\'t know nothin\' elseI tried to show \'em, yeahI tried to show \'em, yeah, yeahYeah, yeah, yeahGone on you with the pick and rollYoung LaFlame, he in sicko modeWoo, made this here with all the ice on in the boothAt the gate outside, when they pull up, they get me looseYeah, Jump Out boys, that\'s Nike boys, hoppin\' out coupesThis shit way too big, when we pull up give me the loot(Gimme the loot!)Was off the Remy, had a PapooseHad to hit my old town to duck the newsTwo-four hour lockdown, we made no movesNow it\'s 4AM and I\'m back up poppin\' with the crewI just landed in, Chase B mixes pop like Jamba JuiceDifferent colored chains, think my jeweler really sellin\' fruitsAnd they chokin\', man, know the crackers wish it was a nooseSome-some-some, someone saidTo win the retreat, we all in too deepP-p-playin\' for keeps, don\'t play 